"""t0-H,L
	$ $174.71	$ $173.45
"""

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from copy import copy

In [2]:
path1 = "./data/1/"
path2 = "./data/2/"
path3 = "./data/3/"
path4 = "./data/4/"
path5 = "./data/5/"
path6 = "./data/UpToDate/"

In [3]:
KP = 80
KF = 25

In [4]:
def fix_dollars(apple2):
    new_open = []
    new_close = []
    new_high = []
    new_low = []
    for i in range(len(apple2['Open'])):
        new_open.append(float(apple2['Open'][i][1:]))
        new_close.append(float(apple2['Close/Last'][i][1:]))
        new_high.append(float(apple2['High'][i][1:]))
        new_low.append(float(apple2['Low'][i][1:]))
    

    apple2.drop('Open',axis=1,inplace=True)
    apple2.drop('Close/Last',axis=1,inplace=True)
    apple2.drop('High',axis=1,inplace=True)
    apple2.drop('Low',axis=1,inplace=True)
    

    apple2['Close'] = new_close
    apple2['Open'] = new_open
    apple2['High'] = new_high
    apple2['Low'] = new_low
    
  

In [5]:
def fix_date(apple2):
    month = []
    year = []
    week_start = []
    week_end = []
    flag = False
    for i in range(len(apple2['Date'])):
        month.append(int(apple2['Date'][i][0:2]))
        year.append(int(apple2['Date'][i][6:]))
        today = int(apple2['Date'][i][3:5])
        
        if i == 0:
            tomorrow = int(apple2['Date'][i + 1][3:5])
            if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i + 1][3:5]) - 1 == int(apple2['Date'][i + 2][3:5]) - 2 \
                    == int(apple2['Date'][i + 3][3:5]) - 3 == int(apple2['Date'][i + 4][3:5]) - 4:
                week_start.append(True)
                week_end.append(False)
            elif today + 1 == tomorrow:
                week_start.append(False)
                week_end.append(False)
            else:
                week_start.append(False)
                week_end.append(True)
                flag = True
                continue
        else:
            try:
                tomorrow = int(apple2['Date'][i + 1][3:5])
                if today + 1 == tomorrow:  # I should really be cross referencing with a calendar api, not hacking this together
                    week_start.append(False)
                    week_end.append(False)
                else:
                    week_start.append(False)
                    week_end.append(True)
                    flag = True
                    continue
                if flag:
                    week_start[-1] = True
                    flag = False
            except:
                yesterday = int(apple2['Date'][i - 1][3:5])
                if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i - 1][3:5]) + 1 == int(
                        apple2['Date'][i - 2][3:5]) + 2 \
                        == int(apple2['Date'][i - 3][3:5]) + 3 == int(apple2['Date'][i - 4][3:5]) + 4:
                    week_start.append(False)
                    week_end.append(True)
                elif yesterday + 1 == today:
                    week_start.append(False)
                    week_end.append(False)
                else:
                    week_start.append(True)
                    week_end.append(False)

    apple2.drop('Date', axis=1, inplace=True)

    apple2['Month'] = month
    apple2['Year'] = year
    apple2['WeekStart'] = week_start
    apple2['WeekEnd'] = week_end


In [6]:
def get_targets(df,KF):
    target_percent_inc = []
    target_best_day = []
    target_worst_sell = []
    target_day_high = []
    target_day_low = []
    options = ['t--'+str(i)+'Low' for i in range(1,KF+1)]
    for i in range(len(df['t-0Low'])):
        today = df['t-0High'][i]
        target_day_high.append(today)
        today_worst = df['t-0Low'][i]
        target_day_low.append(today_worst)
        best = np.max([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])
        worst = np.max([df['t--'+str(j)+'High'][i] for j in range(1,KF+1)])
        best_day = np.argmax([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])+1
        target_percent_inc.append(round((best-today)*100/today,0))
        target_worst_sell.append(round((worst-today_worst)*100/today,0))
        target_best_day.append(best_day)
    return target_percent_inc, target_best_day, target_worst_sell,target_day_high,target_day_low

In [7]:
def fix_shift(apple2,KP,KF,updated=False):
    #KP = 50
    #changing KP and KF from 10 to 30 causes accuracy to drop...Why? Possibly because there are now more classes for y
    #KF = 25

    names = ['Volume','Close','Open','High','Low','Month','Year'] 
    cases = pd.DataFrame(apple2.drop('WeekStart',axis=1).drop('WeekEnd',axis=1).values)
    df = pd.concat([cases.shift(i) for i in range(KP-int(updated),-KF-1,-1)],axis=1).join(apple2['WeekStart']).join(apple2['WeekEnd'])
    df.columns = ['t-'+str(i+int(updated))+x for i in range(KP-int(updated),-KF-1,-1) for x in names]+['WeekStart','WeekEnd']
    return df

In [8]:
def fix_normalize(apple2,df):
    for apple_col in list(apple2):
        if apple_col in ['WeekStart','WeekEnd']:
            continue
        MEAN = apple2[apple_col].mean()
        STD = apple2[apple_col].std()
        for df_col in list(df):
            if apple_col in df_col:
                if apple_col =='Month' or apple_col == 'Year':
                    df = df.astype({df_col: 'int32'})
                else:
                    df[df_col]=round((df[df_col]-MEAN)/STD,0)

In [9]:
def drop_future(df,KF):
    for i in range(0,-KF-1,-1):
        for x in ['Month','Year','Volume','Close','Open','High','Low']:
            df.drop('t-'+str(i)+x,axis=1,inplace=True)

In [10]:
def fix_reverse(apple2):
    return apple2.loc[::-1].reset_index(drop=True)

In [11]:
def drop_weekends(df):
    df = df[df['WeekEnd']==False]
    return df.reset_index(drop=True)

In [12]:
apple = pd.read_csv(path5+"aapl_historical.csv")
apple = apple.iloc[:250]
apple2 = apple.copy(deep=True)
fix_dollars(apple2)
apple2 = fix_reverse(apple2)
apple2

,Date,Volume,Close,Open,High,Low
0,05/17/2022,78336250,149.240,148.86,149.7700,146.68
1,05/18/2022,109742900,140.820,146.85,147.3601,139.90
2,05/19/2022,136095600,137.350,139.88,141.6600,136.60
3,05/20/2022,137426100,137.590,139.09,140.7000,132.61
4,05/23/2022,117726300,143.110,137.79,143.2600,137.65
...,...,...,...,...,...,...
245,05/09/2023,45326870,171.770,173.05,173.5400,171.60
246,05/10/2023,53724500,173.555,173.02,174.0300,171.90
247,05/11/2023,49514680,173.750,173.85,174.5900,172.17
248,05/12/2023,45533140,172.570,173.62,174.0600,171.00


In [13]:
fix_date(apple2)
apple2

,Volume,Close,Open,High,Low,Month,Year,WeekStart,WeekEnd
0,78336250,149.240,148.86,149.7700,146.68,5,2022,False,False
1,109742900,140.820,146.85,147.3601,139.90,5,2022,False,False
2,136095600,137.350,139.88,141.6600,136.60,5,2022,False,False
3,137426100,137.590,139.09,140.7000,132.61,5,2022,False,True
4,117726300,143.110,137.79,143.2600,137.65,5,2022,True,False
...,...,...,...,...,...,...,...,...,...
245,45326870,171.770,173.05,173.5400,171.60,5,2023,False,False
246,53724500,173.555,173.02,174.0300,171.90,5,2023,False,False
247,49514680,173.750,173.85,174.5900,172.17,5,2023,False,False
248,45533140,172.570,173.62,174.0600,171.00,5,2023,False,True


In [14]:
df = fix_shift(apple2,KP,KF)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,72433770.0,138.27,136.820,138.590,135.630,6.0,2022.0,False,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,89116840.0,141.66,139.900,141.910,139.770,6.0,2022.0,False,False
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,70207910.0,141.66,142.695,143.490,140.965,6.0,2022.0,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,67315330.0,137.44,142.130,143.422,137.325,6.0,2022.0,False,True
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,66242410.0,139.23,137.460,140.670,136.670,6.0,2022.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,71379650.0,133.41,133.880,134.26,131.44,1.0,2023.0,57809720.0,134.76,132.030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
246,57809720.0,134.76,132.030,134.92,131.66,1.0,2023.0,63646630.0,135.94,134.830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
247,63646630.0,135.94,134.830,137.29,134.13,1.0,2023.0,69672800.0,135.21,136.815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
248,69672800.0,135.21,136.815,138.61,135.03,1.0,2023.0,58280410.0,135.27,134.080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True


In [15]:
def fix_null(df):
    df.dropna(inplace=True)
    return df.reset_index(drop=True)
    

In [16]:
df = fix_null(df)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,78336250.0,149.24,148.860,149.7700,146.68,5.0,2022.0,109742900.0,140.82,146.850,...,2022.0,85250940.0,142.410,141.065,142.9000,140.270,10.0,2022.0,True,False
1,109742900.0,140.82,146.850,147.3601,139.90,5.0,2022.0,136095600.0,137.35,139.880,...,2022.0,99136610.0,143.750,145.490,146.7000,140.610,10.0,2022.0,False,False
2,136095600.0,137.35,139.880,141.6600,136.60,5.0,2022.0,137426100.0,137.59,139.090,...,2022.0,61758340.0,143.860,141.690,144.9492,141.500,10.0,2022.0,False,False
3,137426100.0,137.59,139.090,140.7000,132.61,5.0,2022.0,117726300.0,143.11,137.790,...,2022.0,64521990.0,143.390,143.020,145.8900,142.650,10.0,2022.0,False,False
4,117726300.0,143.11,137.790,143.2600,137.65,5.0,2022.0,104132700.0,140.36,140.805,...,2022.0,86548610.0,147.270,142.870,147.8479,142.649,10.0,2022.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,64727190.0,142.91,147.075,147.3000,141.92,12.0,2022.0,69721090.0,140.94,142.190,...,2023.0,45326870.0,171.770,173.050,173.5400,171.600,5.0,2023.0,True,False
141,69721090.0,140.94,142.190,143.3700,140.00,12.0,2022.0,62128340.0,142.65,142.360,...,2023.0,53724500.0,173.555,173.020,174.0300,171.900,5.0,2023.0,False,False
142,62128340.0,142.65,142.360,143.5200,141.10,12.0,2022.0,76097010.0,142.16,142.340,...,2023.0,49514680.0,173.750,173.850,174.5900,172.170,5.0,2023.0,False,False
143,76097010.0,142.16,142.340,145.5700,140.90,12.0,2022.0,70462650.0,144.49,142.700,...,2023.0,45533140.0,172.570,173.620,174.0600,171.000,5.0,2023.0,False,True


In [17]:
df = drop_weekends(df)

In [18]:
target_percent_inc, target_best_day,target_worst_sell,target_day_high,target_day_low = get_targets(df,KF)
np.average(target_day_high)

148.4264318181818

In [19]:
np.average(target_day_low)

144.63696

In [20]:
fix_normalize(apple2,df)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,0.0,-0.0,-0.0,-0.0,-0.0,5.0,2022.0,1.0,-1.0,-0.0,...,2022.0,0.0,-1.0,-1.0,-1.0,-1.0,10.0,2022.0,True,False
1,1.0,-1.0,-0.0,-0.0,-1.0,5.0,2022.0,3.0,-1.0,-1.0,...,2022.0,1.0,-1.0,-0.0,-0.0,-1.0,10.0,2022.0,False,False
2,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,3.0,-1.0,-1.0,...,2022.0,-1.0,-1.0,-1.0,-1.0,-1.0,10.0,2022.0,False,False
3,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,2.0,-1.0,-1.0,...,2022.0,-1.0,-1.0,-1.0,-1.0,-0.0,10.0,2022.0,False,False
4,1.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,1.0,-1.0,-1.0,...,2022.0,-0.0,-0.0,-0.0,-0.0,-0.0,10.0,2022.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,-0.0,-0.0,-0.0,-0.0,-0.0,12.0,2022.0,-0.0,-0.0,-0.0,...,2023.0,2.0,2.0,2.0,2.0,2.0,5.0,2023.0,False,False
106,-0.0,-1.0,-0.0,-0.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,True,False
107,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,False,False
108,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,0.0,-1.0,-1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,False,False


In [21]:
drop_future(df,KF)
df

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,0.0,-0.0,-0.0,-0.0,-0.0,5.0,2022.0,1.0,-1.0,-0.0,...,2022.0,-0.0,1.0,0.0,1.0,1.0,9.0,2022.0,True,False
1,1.0,-1.0,-0.0,-0.0,-1.0,5.0,2022.0,3.0,-1.0,-1.0,...,2022.0,1.0,1.0,1.0,1.0,1.0,9.0,2022.0,False,False
2,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,3.0,-1.0,-1.0,...,2022.0,2.0,0.0,1.0,1.0,0.0,9.0,2022.0,False,False
3,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,2.0,-1.0,-1.0,...,2022.0,1.0,0.0,0.0,0.0,0.0,9.0,2022.0,False,False
4,1.0,-1.0,-1.0,-1.0,-1.0,5.0,2022.0,1.0,-1.0,-1.0,...,2022.0,4.0,0.0,0.0,-0.0,0.0,9.0,2022.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,-0.0,-0.0,-0.0,-0.0,-0.0,12.0,2022.0,-0.0,-0.0,-0.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
106,-0.0,-1.0,-0.0,-0.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-0.0,1.0,1.0,1.0,1.0,3.0,2023.0,True,False
107,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,4.0,2023.0,False,False
108,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,0.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,4.0,2023.0,False,False


In [22]:
def round_low(y):
    for i in range(len(y)):
        y[i] = round(y[i])

In [23]:
#For better accuracy
def round_target(y,floor):
    for i in range(len(y)):
        curr = y[i]
        if 0<curr<floor:
            y[i] = 1
        elif -floor<curr<0:
            y[i] = -1
        else:
            y[i] = curr//floor
            if y[i] > 3:
                y[i] = 3
            if y[i] < -3:
                y[i] = -3

In [24]:
#finds min percent gains over KF days to double money in 1 year
def double_yearly(KF):
    TRADING_DAYS = 252
    periods = TRADING_DAYS/KF
    perc = (-1+2**(1/periods))*100
    return perc

In [25]:
def round_day(y):
    for i in range(len(y)):
        y[i] = y[i]//5

In [26]:
X = df
y = copy(target_percent_inc)
y_sell = copy(target_worst_sell)
y_day = copy(target_best_day)
round_day(y_day)
round_target(y,double_yearly(KF))
round_target(y_sell,double_yearly(KF))
y_sell

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1.0,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 3.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1,
 1.0,
 1,
 1,
 1.0,
 1.0]

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression

In [28]:
X['WeekEnd']

0      False
1      False
2      False
3      False
4      False
       ...  
105    False
106    False
107    False
108    False
109    False
Name: WeekEnd, Length: 110, dtype: bool

In [29]:
clf_low = MLPClassifier()

In [30]:
clf = MLPClassifier(random_state=None)
clf_day = MLPClassifier(random_state=None)
clf_sell = MLPClassifier(random_state=1)

In [31]:
round_low(target_day_low)
target_day_low

[159,
 153,
 154,
 151,
 149,
 153,
 154,
 151,
 150,
 150,
 145,
 141,
 138,
 144,
 143,
 145,
 139,
 138,
 138,
 134,
 140,
 141,
 142,
 143,
 146,
 149,
 148,
 144,
 149,
 145,
 139,
 136,
 137,
 135,
 140,
 147,
 149,
 147,
 146,
 148,
 147,
 143,
 140,
 147,
 146,
 142,
 140,
 141,
 141,
 144,
 141,
 136,
 131,
 130,
 133,
 130,
 129,
 126,
 128,
 124,
 125,
 125,
 130,
 128,
 130,
 131,
 134,
 135,
 134,
 138,
 140,
 139,
 142,
 143,
 141,
 148,
 151,
 151,
 151,
 150,
 151,
 151,
 153,
 153,
 148,
 147,
 147,
 147,
 145,
 144,
 153,
 151,
 152,
 150,
 148,
 150,
 150,
 152,
 154,
 157,
 158,
 158,
 158,
 156,
 159,
 161,
 164,
 165,
 162,
 160]

In [32]:
target_day_low[-1]

160

In [33]:
low_scores = cross_val_score(clf_low,X,target_day_low,cv=5,error_score='raise')

C:\Users\phill\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [34]:
low_scores

array([0.        , 0.        , 0.04545455, 0.04545455, 0.04545455])

In [35]:
scores = cross_val_score(clf, X, y, cv=5) #noticable improvement in variance and aveerage after adding week start and end
#except for the fact that without random_state=0 it can still be pretty bad...
#I know this is really bad practice, but for now I don't care
scores

array([0.22727273, 0.09090909, 0.22727273, 0.22727273, 0.68181818])

In [36]:
scores_sell = cross_val_score(clf_sell, X, y_sell, cv=5)
#The model is better at predicting the best case than the worst case for some reason
#(If I can buy a daily low and sell a daily high, the model gives me a more accurate prediction for percent increase)
#than if I buy a daily high and sell a daily low
scores_sell

array([0.86363636, 0.86363636, 0.86363636, 0.86363636, 0.81818182])

In [37]:
scores_day = cross_val_score(clf_day, X, y_day, cv=3)#not really any better
scores_day

array([0.13513514, 0.2972973 , 0.27777778])

In [38]:
clf.fit(X,y)
clf_day.fit(X,y_day)
clf_sell.fit(X,y_sell)

MLPClassifier(random_state=1)

In [39]:
clf_low.fit(X,target_day_low)

MLPClassifier()

In [40]:
uptodate = pd.read_csv(path6+"apple.csv")
uptodate

,Date,Close/Last,Volume,Open,High,Low
0,05/23/2023,$171.56,50747260,$173.13,$173.3794,$171.275
1,05/22/2023,$174.20,43570930,$173.98,$174.71,$173.45
2,05/19/2023,$175.16,55809480,$176.39,$176.39,$174.94
3,05/18/2023,$175.05,65496660,$173.00,$175.24,$172.58
4,05/17/2023,$172.69,57951600,$171.71,$172.925,$170.4201
...,...,...,...,...,...,...
119,11/30/2022,$148.03,111380900,$141.395,$148.72,$140.55
120,11/29/2022,$141.17,83763800,$144.29,$144.81,$140.355
121,11/28/2022,$144.22,69346520,$145.14,$146.64,$143.38
122,11/25/2022,$148.11,35195860,$148.305,$148.88,$147.12


In [41]:
u2 = uptodate.copy(deep=True)
fix_dollars(u2)
u2 = u2.loc[::-1].reset_index(drop=True)
u2

,Date,Volume,Close,Open,High,Low
0,11/23/2022,58301400,151.07,149.450,151.8300,149.3400
1,11/25/2022,35195860,148.11,148.305,148.8800,147.1200
2,11/28/2022,69346520,144.22,145.140,146.6400,143.3800
3,11/29/2022,83763800,141.17,144.290,144.8100,140.3550
4,11/30/2022,111380900,148.03,141.395,148.7200,140.5500
...,...,...,...,...,...,...
119,05/17/2023,57951600,172.69,171.710,172.9250,170.4201
120,05/18/2023,65496660,175.05,173.000,175.2400,172.5800
121,05/19/2023,55809480,175.16,176.390,176.3900,174.9400
122,05/22/2023,43570930,174.20,173.980,174.7100,173.4500


In [42]:
fix_date(u2)

In [43]:
u2

,Volume,Close,Open,High,Low,Month,Year,WeekStart,WeekEnd
0,58301400,151.07,149.450,151.8300,149.3400,11,2022,False,True
1,35195860,148.11,148.305,148.8800,147.1200,11,2022,False,True
2,69346520,144.22,145.140,146.6400,143.3800,11,2022,True,False
3,83763800,141.17,144.290,144.8100,140.3550,11,2022,False,False
4,111380900,148.03,141.395,148.7200,140.5500,11,2022,False,True
...,...,...,...,...,...,...,...,...,...
119,57951600,172.69,171.710,172.9250,170.4201,5,2023,False,False
120,65496660,175.05,173.000,175.2400,172.5800,5,2023,False,False
121,55809480,175.16,176.390,176.3900,174.9400,5,2023,False,True
122,43570930,174.20,173.980,174.7100,173.4500,5,2023,True,False


In [44]:
dfu = fix_shift(u2,KP,0,updated=True)
dfu

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,58301400.0,151.07,149.450,151.8300,149.3400,11.0,2022.0,False,True
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,35195860.0,148.11,148.305,148.8800,147.1200,11.0,2022.0,False,True
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,69346520.0,144.22,145.140,146.6400,143.3800,11.0,2022.0,True,False
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,83763800.0,141.17,144.290,144.8100,140.3550,11.0,2022.0,False,False
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,111380900.0,148.03,141.395,148.7200,140.5500,11.0,2022.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,66435140.0,142.53,140.305,143.16,140.30,1.0,2023.0,65799350.0,141.86,140.890,...,2023.0,57951600.0,172.69,171.710,172.9250,170.4201,5.0,2023.0,False,False
120,65799350.0,141.86,140.890,142.43,138.81,1.0,2023.0,54105070.0,143.96,143.170,...,2023.0,65496660.0,175.05,173.000,175.2400,172.5800,5.0,2023.0,False,False
121,54105070.0,143.96,143.170,144.25,141.90,1.0,2023.0,70555840.0,145.93,143.155,...,2023.0,55809480.0,175.16,176.390,176.3900,174.9400,5.0,2023.0,False,True
122,70555840.0,145.93,143.155,147.23,143.08,1.0,2023.0,64015270.0,143.00,144.955,...,2023.0,43570930.0,174.20,173.980,174.7100,173.4500,5.0,2023.0,True,False


In [45]:
dfu.dropna(inplace=True)
dfu = dfu.reset_index(drop=True)
dfu

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,58301400.0,151.07,149.450,151.8300,149.340,11.0,2022.0,35195860.0,148.11,148.305,...,2023.0,73938290.0,159.280,157.320,159.4000,156.5400,3.0,2023.0,False,False
1,35195860.0,148.11,148.305,148.8800,147.120,11.0,2022.0,69346520.0,144.22,145.140,...,2023.0,75701810.0,157.830,159.300,162.1400,157.8100,3.0,2023.0,False,False
2,69346520.0,144.22,145.140,146.6400,143.380,11.0,2022.0,83763800.0,141.17,144.290,...,2023.0,67622060.0,158.930,158.830,161.5501,157.6800,3.0,2023.0,False,False
3,83763800.0,141.17,144.290,144.8100,140.355,11.0,2022.0,111380900.0,148.03,141.395,...,2023.0,59256340.0,160.250,158.860,160.3400,157.8500,3.0,2023.0,False,True
4,111380900.0,148.03,141.395,148.7200,140.550,11.0,2022.0,71250420.0,148.31,148.210,...,2023.0,52390270.0,158.280,159.940,160.7700,157.8700,3.0,2023.0,True,False
5,71250420.0,148.31,148.210,149.1300,146.610,12.0,2022.0,65447450.0,147.81,145.960,...,2023.0,45992150.0,157.650,157.970,158.4900,155.9800,3.0,2023.0,False,False
6,65447450.0,147.81,145.960,148.0000,145.650,12.0,2022.0,68826440.0,146.63,147.770,...,2023.0,51305690.0,160.770,159.370,161.0500,159.3500,3.0,2023.0,False,False
7,68826440.0,146.63,147.770,150.9199,145.770,12.0,2022.0,64727190.0,142.91,147.075,...,2023.0,49501690.0,162.360,161.530,162.4700,161.2710,3.0,2023.0,False,False
8,64727190.0,142.91,147.075,147.3000,141.920,12.0,2022.0,69721090.0,140.94,142.190,...,2023.0,68749790.0,164.900,162.440,165.0000,161.9100,3.0,2023.0,False,True
9,69721090.0,140.94,142.190,143.3700,140.000,12.0,2022.0,62128340.0,142.65,142.360,...,2023.0,56976190.0,166.170,164.270,166.2900,164.2200,4.0,2023.0,True,False


In [46]:
fix_normalize(apple2,dfu)
dfu

,t-80Volume,t-80Close,t-80Open,t-80High,t-80Low,t-80Month,t-80Year,t-79Volume,t-79Close,t-79Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,-1.0,0.0,-0.0,-0.0,0.0,11.0,2022.0,-2.0,-0.0,-0.0,...,2023.0,-0.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
1,-2.0,-0.0,-0.0,-0.0,-0.0,11.0,2022.0,-0.0,-1.0,-0.0,...,2023.0,-0.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
2,-0.0,-1.0,-0.0,-0.0,-0.0,11.0,2022.0,0.0,-1.0,-0.0,...,2023.0,-0.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
3,0.0,-1.0,-0.0,-1.0,-1.0,11.0,2022.0,2.0,-0.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,True
4,2.0,-0.0,-1.0,-0.0,-1.0,11.0,2022.0,-0.0,-0.0,-0.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,True,False
5,-0.0,-0.0,-0.0,-0.0,-0.0,12.0,2022.0,-0.0,-0.0,-0.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
6,-0.0,-0.0,-0.0,-0.0,-0.0,12.0,2022.0,-0.0,-0.0,-0.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
7,-0.0,-0.0,-0.0,-0.0,-0.0,12.0,2022.0,-0.0,-1.0,-0.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,False
8,-0.0,-1.0,-0.0,-0.0,-1.0,12.0,2022.0,-0.0,-1.0,-1.0,...,2023.0,-0.0,1.0,1.0,1.0,1.0,3.0,2023.0,False,True
9,-0.0,-1.0,-1.0,-1.0,-1.0,12.0,2022.0,-1.0,-1.0,-1.0,...,2023.0,-1.0,1.0,1.0,1.0,1.0,4.0,2023.0,True,False


In [47]:
point = np.array(dfu.iloc[-2]).reshape(1,-1)
point

array([[-0.0, -0.0, -1.0, -0.0, -0.0, 1.0, 2023.0, -1.0, -1.0, -0.0,
        -1.0, -0.0, 1.0, 2023.0, -0.0, -1.0, -1.0, -1.0, -1.0, 1.0,
        2023.0, 0.0, -0.0, -1.0, -0.0, -1.0, 2.0, 2023.0, 2.0, 0.0, -0.0,
        -0.0, -0.0, 2.0, 2023.0, 3.0, 0.0, -0.0, 0.0, -0.0, 2.0, 2023.0,
        -0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2023.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        2.0, 2023.0, -1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2023.0, -1.0, 0.0,
        0.0, 0.0, 0.0, 2.0, 2023.0, -1.0, 0.0, -0.0, -0.0, 0.0, 2.0,
        2023.0, -1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2023.0, -1.0, 0.0, 0.0,
        0.0, 0.0, 2.0, 2023.0, -0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2023.0,
        -0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2023.0, -1.0, 0.0, 0.0, 0.0, 0.0,
        2.0, 2023.0, -1.0, -0.0, 0.0, -0.0, 0.0, 2.0, 2023.0, -1.0, -0.0,
        -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0, 0.0, -0.0, -0.0, 2.0,
        2023.0, -1.0, -0.0, -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0,
        -0.0, -0.0, -0.0, 2.0, 2023.0, -1.0, -0.0, -0.0, -0.0, -0.0, 2.0

In [48]:
clf.predict(point)#Now it's saying apple will go down.
#With random state = 0 and 3 cross val scores approx 70%, predict apple will go up by less than ~8%
#With random state = 1 and 2 good scores 1 bad, predict apple will go down by less than 8%
# (given tomorrow's high, the highest low of the next 25 days will be down by less than 8%)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([1.])

In [49]:
clf_sell.predict(point)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([1.])

In [50]:
clf_day.predict(point)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([4], dtype=int64)

In [51]:
clf_low.predict(point)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([148])